In [1]:
!pip install wfdb
import io
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath
import wfdb
import urllib.request
import datetime

     |████████████████████████████████| 112kB 5.4MB/s 
     |████████████████████████████████| 6.6MB 5.9MB/s 
     |████████████████████████████████| 163kB 43.3MB/s 


In [2]:
from google.colab import files
uploaded = files.upload()

Saving df_TS_exists_withoutTEMP_overlapcount.csv to df_TS_exists_withoutTEMP_overlapcount.csv


In [4]:
df_csvdata = pd.read_csv(io.BytesIO(uploaded['df_TS_exists_withoutTEMP_overlapcount.csv']))
# Dataset is now stored in a Pandas Dataframe
print ('shape of original dataframe from CSV : ', df_csvdata.shape)

shape of original dataframe from CSV :  (4653, 8)


In [7]:
df_csvdata = df_csvdata[(df_csvdata['sig_exists'] == 1) & (df_csvdata['timeoverlap'] == 1)]
df_csvdata['first24']=''; 
df_csvdata['first12']='';
df_csvdata['first6']='';
df_csvdata['6around_onset']='';
df_csvdata['8around_onset']='';
df_csvdata['entire_stay']='';
df_csvdata['gap']='';
print ('Shape of dataframe with only ICU stays for whom the signals and time overlap exists : ',df_csvdata.shape )

Shape of dataframe with only ICU stays for whom the signals and time overlap exists :  (1655, 15)


In [ ]:
from collections import namedtuple
from urllib.request import urlopen

df_ts_records_columns = ['RECORD','TIME','HR', 'SPO2','ABPSYS','ABPDIAS','ABPMEAN','RESP'] 

for index, row in df_csvdata.iterrows():
    print(row['subject_id']);
    try:
      df_ts_records.drop(df_ts_records.index,inplace=True)
    except:
      print('MAIN DF does not exits')

    df_ts_records = pd.DataFrame(columns=df_ts_records_columns);

    wdb_dir_path = 'mimic3wdb/matched/p'+ str(row['subject_id']).zfill(6)[:2] + '/p' + str(row['subject_id']).zfill(6) + '/';
    wdb_path_toAllRecords = 'https://archive.physionet.org/physiobank/database/'+ wdb_dir_path + 'RECORDS';
    #print(wdb_path_toAllRecords)
    wdb_records =  urllib.request.urlopen(wdb_path_toAllRecords);   
    #print('SUCCESS')
    count_overlap = 0;
    gap = ''; 
    for lines in wdb_records.readlines():
      record = lines.decode("utf-8"); 
      record = str(record).rstrip()
      #print (record[-1:])
      if record[-1:] == 'n':
        #print(record);
        #print (wdb_dir_path);
        consider_record =0 ;
        record = str(record).rstrip()

        try:
          signals =''
          fields = ''
          #print(record)
          #print(wdb_dir_path)
          signals,fields = wfdb.rdsamp(record, pn_dir=wdb_dir_path) ; 
          list_sig_name = [item.upper().replace(' ','') for item in fields['sig_name']]
          sig_exist_1 = all(x in list_sig_name for x in ['HR', 'SPO2','ABPSYS','ABPDIAS','ABPMEAN','RESP']);  #%SpO2
          sig_exist_2 = all(x in list_sig_name for x in ['HR', '%SPO2','ABPSYS','ABPDIAS','ABPMEAN','RESP']); 
          if ((sig_exist_1 == True) or (sig_exist_2 == True)) :
            consider_record = 1
          else:
            consider_record = 0
          record_starttime = datetime.datetime.combine(fields['base_date'] ,fields['base_time'] ) ;
          if  '%.3f'%(fields['fs']) == '1.000' :
            record_endtime = record_starttime + datetime.timedelta(seconds= (fields['sig_len']-1)) ;
          elif '%.3f'%(fields['fs'])== '0.017' :
            record_endtime = record_starttime + datetime.timedelta(minutes = (fields['sig_len']-1)) ;
          else : 
            print('ERROR IN SAMPLING');
            print(record);
            print (wdb_dir_path);
          #Caculate if we have a recording for the time of icu stay
          Range = namedtuple('Range', ['start', 'end'])
          r1 = Range(start= datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S'), end= datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S'))
          r2 = Range(start= record_starttime, end = record_endtime)
          latest_start = max(r1.start, r2.start)
          earliest_end = min(r1.end, r2.end)
          delta = (earliest_end - latest_start).days + 1
          df_row_idx = df_ts_records.shape[0] ;
          if ((delta >= 0 ) & (consider_record ==1)) :
            #print('RECORD EXISTS FOR THE ICU STAYS WITH THE SIGNALS NEEDED : ', row['subject_id'])
            #df_csvdata.loc[index,'timeoverlap'] = 1;
            #todo : adding new dataframe, exatracting required signals, computing avergage for per sminute values in case of per second sampling frequency
            for i in fields['sig_name']:
              if i.upper().replace(' ','') == 'HR':
                idx_HR='';
                idx_HR = fields['sig_name'].index(i);
              elif (( i.upper().replace(' ','') == 'SPO2') or (i.upper().replace(' ','') =='%SPO2')):
                idx_SPO2 = '';
                idx_SPO2 = fields['sig_name'].index(i);
              elif i.upper().replace(' ','') == 'ABPSYS' :
                idx_ABPSYS = '';
                idx_ABPSYS = fields['sig_name'].index(i);
              elif i.upper().replace(' ','') == 'ABPDIAS' :
                idx_ABPDIAS = '';
                idx_ABPDIAS = fields['sig_name'].index(i);
              elif i.upper().replace(' ','') == 'ABPMEAN' :
                idx_ABPMEAN = '';
                idx_ABPMEAN = fields['sig_name'].index(i);
              elif i.upper().replace(' ','') == 'RESP' :
                idx_RESP = '';
                idx_RESP = fields['sig_name'].index(i);
                
            if count_overlap == 0 : 
              if record_starttime > (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ):
                #print('inserting nulls between icu intime and record start time')
                minutes_to_insert_start = record_starttime - (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') )
                #print('minutes_to_insert_start:  ', minutes_to_insert_start)
                duration_in_s = minutes_to_insert_start.total_seconds()
                minutes_to_insert_start = divmod(duration_in_s, 60)[0] - 1 
                gap = gap + ',' + str(minutes_to_insert_start)
                try:
                  df_ts_records_time_temp_start.drop(df_ts_records_time_temp_start.index,  inplace=True)
                except :
                  print( 'df_ts_records_time_temp_start does not exist')
                df_ts_records_time_temp_start = pd.DataFrame(columns=df_ts_records_columns)
                if '%.3f'%(fields['fs'])== '0.017' :
                  df_ts_records_time_temp_start['TIME'] = pd.date_range((datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(minutes=1), 
                                                              periods = minutes_to_insert_start, freq='1min'); 
                elif '%.3f'%(fields['fs'])== '1.000' :
                  df_ts_records_time_temp_start['TIME'] = pd.date_range((datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(seconds=1), 
                                                              periods = (duration_in_s-1), freq='S');  
                df_ts_records = df_ts_records.append(df_ts_records_time_temp_start, ignore_index=True);
                gap = gap + ',start:' + str(minutes_to_insert_start)

              try:
                df_ts_records_temp.drop(df_ts_records_temp.index,  inplace=True)
              except:
                print( 'df_ts_records_time_temp_start does not exist')
              df_ts_records_temp = pd.DataFrame(columns=df_ts_records_columns)
              df_ts_records_temp['HR']= signals[:,idx_HR ] 
              df_ts_records_temp['SPO2']= signals[:,idx_SPO2 ] 
              df_ts_records_temp['ABPSYS']= signals[:,idx_ABPSYS ] 
              df_ts_records_temp['ABPDIAS']= signals[:,idx_ABPDIAS ] 
              df_ts_records_temp['ABPMEAN']= signals[:,idx_ABPMEAN ] 
              df_ts_records_temp['RESP']= signals[:,idx_RESP ] 
              if '%.3f'%(fields['fs'])== '0.017' :
                df_ts_records_temp['TIME'] = pd.date_range(record_starttime, periods=fields['sig_len'], freq='1min'); 
              elif '%.3f'%(fields['fs'])== '1.000' :
                df_ts_records_temp['TIME'] = pd.date_range(record_starttime, periods=fields['sig_len'], freq='S'); 
              df_ts_records_temp.TIME = pd.to_datetime(df_ts_records_temp.TIME)
              df_ts_records = df_ts_records.append(df_ts_records_temp, ignore_index=True);


            else:
              if record_starttime < (datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S') ) :
                last_Record_time = df_ts_records.loc[(df_row_idx-1),'TIME']
                #print('main DF last time record: ',last_Record_time )
                minutes_to_insert = record_starttime - last_Record_time
                duration_in_s = minutes_to_insert.total_seconds()
                minutes_to_insert = divmod(duration_in_s, 60)[0] - 1
                #print ('minutes_to_insert:  ', minutes_to_insert);
                try:
                  df_ts_records_time_temp.drop(df_ts_records_time_temp.index, inplace= True);
                  df_ts_records_temp.drop(df_ts_records_temp.index, inplace=True);
                except:
                  print ('df_ts_records_temp and df_ts_records_time_temp does not exits')
                df_ts_records_time_temp = pd.DataFrame(columns=df_ts_records_columns)
                if '%.3f'%(fields['fs'])== '0.017' :
                  df_ts_records_time_temp['TIME'] = pd.date_range(last_Record_time + datetime.timedelta(minutes=1), 
                                                              periods=minutes_to_insert, freq='1min'); 
                elif '%.3f'%(fields['fs'])== '1.000' :
                  df_ts_records_time_temp['TIME'] = pd.date_range(last_Record_time + datetime.timedelta(seconds=1), 
                                                              periods=(duration_in_s-1), freq='S'); 
                #print ('df_ts_records_time_temp:')
                #print (df_ts_records_time_temp)
                df_ts_records = df_ts_records.append(df_ts_records_time_temp, ignore_index=True);
                gap = gap + ',mid:' + str(minutes_to_insert)
              
                df_ts_records_temp = pd.DataFrame(columns=df_ts_records_columns)
                df_ts_records_temp['HR']= signals[:,idx_HR ] 
                df_ts_records_temp['SPO2']= signals[:,idx_SPO2 ] 
                df_ts_records_temp['ABPSYS']= signals[:,idx_ABPSYS ] 
                df_ts_records_temp['ABPDIAS']= signals[:,idx_ABPDIAS ] 
                df_ts_records_temp['ABPMEAN']= signals[:,idx_ABPMEAN ] 
                df_ts_records_temp['RESP']= signals[:,idx_RESP ] 
                if '%.3f'%(fields['fs'])== '0.017' :
                  df_ts_records_temp['TIME'] = pd.date_range(record_starttime, periods=fields['sig_len'], freq='1min'); 
                elif  '%.3f'%(fields['fs'])== '1.000' :
                  df_ts_records_temp['TIME'] = pd.date_range(record_starttime, periods=fields['sig_len'], freq='S'); 
                df_ts_records_temp.TIME = pd.to_datetime(df_ts_records_temp.TIME)
                df_ts_records = df_ts_records.append(df_ts_records_temp, ignore_index=True);

        
            df_ts_records['RECORD'] = record

            """
            if '%.3f'%(fields['fs'])== '1.000' :
              start_idx = 0;
              df_ts_records_new = pd.DataFrame(columns=df_ts_records_columns);
              #print('length of new df  '  , df_ts_records_new.shape[0] )
              for index, rows in df_ts_records.iterrows():
                if start_idx >= df_ts_records.shape[0]:
                  exit;
                else: 
                  #print(df_ts_records.iloc[start_idx: (start_idx+60), 2:8])
                  array = np.array( df_ts_records.iloc[start_idx: (start_idx+60), 2:8].mean(axis=0))
                  #print('printing array of average')
                  #print (array)

                  current_index = df_ts_records_new.shape[0]
                  df_ts_records_new.loc[current_index ,'HR']= array[0]
                  df_ts_records_new.loc[current_index,'SPO2']= array[1]
                  df_ts_records_new.loc[current_index,'ABPSYS']= array[2]
                  df_ts_records_new.loc[current_index,'ABPDIAS']= array[3]
                  df_ts_records_new.loc[current_index,'ABPMEAN']= array[4]
                  df_ts_records_new.loc[current_index,'RESP']= array[5]

                  #print(df_ts_records_new)
                  #print('next average')
                  start_idx = start_idx+60;
                  #print('start index :: ' , start_idx)

              df_ts_records_new['TIME'] = pd.date_range(df_ts_records.loc[0,'TIME'], periods= df_ts_records_new.shape[0], freq='1min'); 
              df_ts_records_new.TIME = pd.to_datetime(df_ts_records_new.TIME)
              #print(df_ts_records_new)
              df_ts_records.drop(df_ts_records.index, inplace=True);
              df_ts_records = pd.DataFrame(columns=df_ts_records_columns)
              df_ts_records = df_ts_records.append(df_ts_records_new, ignore_index=True);
            """
            #print('Appended dataframe')
            #print(df_ts_records)
            FS = '%.3f'%(fields['fs'])
            count_overlap = count_overlap +1


          else:            
            print('RECORD DOES NOT EXISTS FOR THE ICU STAYS WITH THE SIGNALS NEEDED : ', row['subject_id'])
              #df_csvdata.loc[index,'timeoverlap'] = 0;

        except ValueError:
          print('Error occured while reading waveform: ', record);


    #print((datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=24))
    try:
      last_record_idx = df_ts_records.shape[0] - 1
      all_records_end_time = df_ts_records.loc[last_record_idx,'TIME']
      if (all_records_end_time < (datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S') ) ):
        print('INSERTING NULLS AT THE END')
        try:
          df_ts_records_time_temp_end.drop(df_ts_records_time_temp_end.index, inplace=True)
        except:
          print('df_ts_records_time_temp_end does not exists')
        #print('main DF last time record: ',last_Record_time )
        minutes_to_insert_end = (datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S') ) - all_records_end_time
        duration_in_s = minutes_to_insert_end.total_seconds()
        minutes_to_insert_end = divmod(duration_in_s, 60)[0] - 1

        df_ts_records_time_temp_end = pd.DataFrame(columns=df_ts_records_columns)
        if FS == '0.017' :
          df_ts_records_time_temp_end['TIME'] = pd.date_range(all_records_end_time + datetime.timedelta(minutes=1), 
                                                              periods=minutes_to_insert_end, freq='1min'); 
        elif FS == '1.000' :
          df_ts_records_time_temp_end['TIME'] = pd.date_range(all_records_end_time + datetime.timedelta(seconds=1), 
                                                              periods=(duration_in_s-1), freq='S'); 
        df_ts_records = df_ts_records.append(df_ts_records_time_temp_end, ignore_index=True);
        gap = gap + ',end:' + str(minutes_to_insert_end)

      
      df_csvdata.loc[index,'gap'] = gap;
      total_rows_first24hour = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=24) )].shape[0]
      total_rows_first24hour_notNAN = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=24) )].dropna().shape[0]
      percent_notNANdata_24 = round(total_rows_first24hour_notNAN / total_rows_first24hour , 2) * 100 
      if percent_notNANdata_24 >= 80:
        print('selected record for first 24 : ', row['subject_id'])
        df_csvdata.loc[index,'first24'] = 1;
      else:
        print('not selected for first 24: ', row['subject_id'])

      total_rows_first12hour = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=12) )].shape[0]
      total_rows_first12hour_notNAN = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=12) )].dropna().shape[0]
      percent_notNANdata_12 = round(total_rows_first12hour_notNAN / total_rows_first12hour , 2) * 100
      if percent_notNANdata_12 >= 80:
        print('selected record for first 12: ', row['subject_id'])
        df_csvdata.loc[index,'first12'] = 1;
      else:
        print('not selected for first 12: ', row['subject_id'])

      total_rows_first6hour = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=6) )].shape[0]
      total_rows_first6hour_notNAN = df_ts_records[(df_ts_records['TIME'] <= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=6) )].dropna().shape[0]
      percent_notNANdata_6 = round(total_rows_first6hour_notNAN / total_rows_first6hour , 2) * 100 
      if percent_notNANdata_6 >= 80:
        print('selected record for first 6 : ', row['subject_id'])
        df_csvdata.loc[index,'first6'] = 1;
      else:
        print('not selected for first 6: ', row['subject_id'])
      
      total_rows_entire_stay = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') )  )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S') )  )].shape[0]
      total_rows_entire_stay_notNAN = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['intime'],'%Y-%m-%d %H:%M:%S') )  )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['outtime'],'%Y-%m-%d %H:%M:%S') )  )].dropna().shape[0]
      percent_notNANdata_entire_stay = round(total_rows_entire_stay_notNAN / total_rows_entire_stay , 2) * 100 
      if percent_notNANdata_entire_stay >= 80:
        print('selected record for entire icu stay : ', row['subject_id'])
        df_csvdata.loc[index,'entire_stay'] = 1;
      else:
        print('not selected for entire icu stay : ', row['subject_id'])

      
      total_rows_6around_onset = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) - datetime.timedelta(hours=6) )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=6) )].shape[0]
      total_rows_6around_onset_notNAN = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) - datetime.timedelta(hours=6) )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=6) )].dropna().shape[0]
      percent_notNANdata_6around_onset = round(total_rows_6around_onset_notNAN / total_rows_6around_onset , 2) * 100 
      if percent_notNANdata_6around_onset >= 80:
        print('selected record for 6 around sepsis onsettime : ', row['subject_id'])
        df_csvdata.loc[index,'6around_onset'] = 1;
      else:
        print('not selected for 6 around sepsis onsettime : ', row['subject_id'])

      
      total_rows_8around_onset = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) - datetime.timedelta(hours=8) )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=8) )].shape[0]
      total_rows_8around_onset_notNAN = df_ts_records[(df_ts_records['TIME'] >= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) - datetime.timedelta(hours=8) )
                                & (df_ts_records['TIME'] <= (datetime.datetime.strptime(row['sepsis_onsettime'],'%Y-%m-%d %H:%M:%S') ) + datetime.timedelta(hours=8) )].dropna().shape[0]
      percent_notNANdata_8around_onset = round(total_rows_8around_onset_notNAN / total_rows_8around_onset , 2) * 100 
      if percent_notNANdata_8around_onset >= 80:
        print('selected record for 8 around sepsis onsettime : ', row['subject_id'])
        df_csvdata.loc[index,'8around_onset'] = 1;
      else:
        print('not selected for 8 around sepsis onsettime : ', row['subject_id'])

      

    except:
      print('Error occured while reading waveform for patient: ', row['subject_id'])

In [10]:

print('count for entire icu: ', df_csvdata[(df_csvdata['entire_stay'] == 1)  ].shape[0]);
print('count for first 24: ', df_csvdata[(df_csvdata['first24'] == 1) ].shape[0]);
print('count for first 12: ', df_csvdata[(df_csvdata['first12'] == 1) ].shape[0]);
print('count for first 6: ', df_csvdata[( df_csvdata['first6'] == 1) ].shape[0]);
print('count for 6 around onset: ', df_csvdata[(df_csvdata['6around_onset'] == 1) ].shape[0]);
print('count for 8 around onset: ', df_csvdata[(df_csvdata['8around_onset'] == 1) ].shape[0]);


count for entire icu:  239
count for first 24:  404
count for first 12:  343
count for first 6:  262
count for 6 around onset:  120
count for 8 around onset:  144


In [11]:
df_csvdata.to_csv ('df_TS_exists_withoutTEMP_MissingValues.csv', sep=',', index = False, header=True);

from google.colab import files
files.download('df_TS_exists_withoutTEMP_MissingValues.csv')
